In [70]:
import requests
import base64
import datetime
from urllib.parse import urlencode
import pprint
import pandas as pd

In [21]:
client_id="8c6548155829445bb3ffb3e26cf3a501"
client_secret="15d8511764804849ba1381758b502733"

In [22]:
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expire=True
    client_id=None
    client_secret=None
    base_url="https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        self.client_id=client_id
        self.client_secret=client_secret
    
    def get_client_credentials(self):
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret == None or client_id == None:
            raise Exception (" Please set client_id and client_secret")
            
        "returns as base64"
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    def get_token_header(self):
        client_creds_b64=self.get_client_credentials()
        return {"Authorization":f"Basic {client_creds_b64}"}
    
    def get_token_data(self):
        return {"grant_type":"client_credentials"}
    
    def perform_auth(self):
        base_url=self.base_url
        token_data=self.get_token_data()
        token_header=self.get_token_header()
        r=requests.post(base_url,data=token_data,headers=token_header)
        
        if r.status_code not in range(200,299):
            return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        access_token=token_response_data["access_token"]
        self.access_token=access_token
        expires_in = token_response_data["expires_in"] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        did_expire = expires < now
        return True
        
     

In [23]:
spotify=SpotifyAPI(client_id,client_secret)

In [24]:
spotify.perform_auth()

True

In [25]:
access_token=spotify.access_token
access_token

'BQCjElT9IaMuaNcfxteeZRw2pjdHR4siNIhowHDhZnOw7WUnxswG-20GtljgJIfo6pD8oeSndApiWd8T3Xs'

In [78]:
artists=["Arijit Singh","Tanishk Bagchi","Neha Kakkar","Badshah","Shreya ghoshal"]
artist_details=pd.DataFrame(columns=["Name","id","Popularity","Followers","Genres"])

for i in artists:
    endpoint="https://api.spotify.com/v1/search"
    headers={"Authorization":f"Bearer {access_token}"}
    data=urlencode({"q":i,"type":"artist","market":"IN","offset":0,"limit":50})
    lookup_url=f"{endpoint}?{data}"
    r=requests.get(lookup_url,headers=headers)
    result=r.json()
    output=result['artists']["items"][0]
    position=len(artist_details)
    artist_details.loc[position]=[output["name"],output["id"],output["popularity"],output["followers"]["total"],output["genres"]]

artist_details

,Name,id,Popularity,Followers,Genres
0,Arijit Singh,4YRxDV8wJFPHPTeXepOstw,83,15590950,"[desi pop, filmi, modern bollywood]"
1,Tanishk Bagchi,4f7KfxeHq9BiylGmyXepGt,76,330764,"[desi hip hop, desi pop, filmi, modern bollywood]"
2,Neha Kakkar,5f4QpKfy7ptCHwTqspnSJI,76,10619019,"[desi hip hop, desi pop, filmi, modern bollywood]"
3,Badshah,0y59o4v8uw5crbN9M3JiL1,75,7377134,"[desi hip hop, desi pop, haryanvi pop, modern ..."
4,Shreya Ghoshal,0oOet2f43PA68X5RxKobEy,74,4434266,"[desi hip hop, desi pop, filmi, modern bollywo..."


In [79]:
artist_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        5 non-null      object
 1   id          5 non-null      object
 2   Popularity  5 non-null      object
 3   Followers   5 non-null      object
 4   Genres      5 non-null      object
dtypes: object(5)
memory usage: 240.0+ bytes
